In [2]:
import pandas as pd
import numpy as np
from scipy.signal import butter, lfilter, welch

In [3]:
# Function definitions
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data, axis=0)
    return y

def calculate_band_power(data, fs, band):
    freqs, psd = welch(data, fs, nperseg=256)
    band_power = np.trapz(psd[(freqs >= band[0]) & (freqs <= band[1])], freqs[(freqs >= band[0]) & (freqs <= band[1])])
    return band_power

In [22]:

# Load the CSV file without headers with tab delimiter
openbci_data_path = 'k1d2.csv'
try:
    data = pd.read_csv(openbci_data_path, header=None)
    print("CSV file loaded successfully.")
    print("Data shape before slicing:", data.shape)
    print("Sample data:")
    print(data.head())
except FileNotFoundError:
    print("CSV file not found. Please check the file path.")
    exit()
except ValueError as ve:
    print(f"Error reading CSV file: {ve}")
    exit()

CSV file loaded successfully.
Data shape before slicing: (4468, 33)
Sample data:
    0             1             2             3             4            5   \
0  0.0   8343.325060   7442.102723  -6226.905433   7605.941010  1144.923406   
1  2.0  16653.726000  14852.488321 -12424.015990  15217.335846  2216.086056   
2  4.0  16677.262387  14873.945996 -12443.439656  15188.949131  2255.983920   
3  6.0  16651.781398  14851.728362 -12424.261859  15203.902448  2225.451437   
4  8.0  16635.665791  14834.986905 -12406.313408  15185.059927  2227.105466   

             6             7             8             9   ...    23    24  \
0   6936.394505   8348.063630   8461.185808   9734.832911  ...  14.0  40.0   
1  13838.814359  16757.594557  16988.018690  18230.306295  ...   0.0   0.0   
2  13862.462504  16720.557716  16954.826350  18202.634836  ...   0.0   0.0   
3  13840.401332  16738.886146  16970.919606  18218.951609  ...   0.0   0.0   
4  13822.810510  16768.345746  17009.141089  18194.275

In [23]:
# Drop the first column (timer) and select the next 16 columns
data = data.iloc[:, 1:17]
print("Data shape after dropping timer and selecting 16 columns:", data.shape)

Data shape after dropping timer and selecting 16 columns: (4468, 16)


In [24]:
# Filter parameters
fs = 125  # Sampling frequency

# Define frequency bands
alpha_band = (8, 13)
beta_band = (13, 30)

# Channel names according to the 10-20 system
channel_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6']

# Preprocess and calculate band power for each channel
alpha_powers = []
beta_powers = []

In [25]:
for i, channel in enumerate(data.T.values):
    print(f"Processing channel {channel_names[i]}")
    print(f"Original channel type: {type(channel)}")
    
    # Ensure channel is a numpy array
    if isinstance(channel, np.ndarray):
        print(f"Original channel shape: {channel.shape}")
    else:
        print("Error: Channel is not a numpy array")
        continue
    
    # Apply bandpass filter
    filtered_data = bandpass_filter(channel, 1.0, 30.0, fs)
    print(f"Filtered data shape: {filtered_data.shape}")

    # Calculate alpha and beta band powers
    alpha_power = calculate_band_power(filtered_data, fs, alpha_band)
    beta_power = calculate_band_power(filtered_data, fs, beta_band)
    
    alpha_powers.append(alpha_power)
    beta_powers.append(beta_power)

Processing channel Fp1
Original channel type: <class 'numpy.ndarray'>
Original channel shape: (4468,)
Filtered data shape: (4468,)
Processing channel Fp2
Original channel type: <class 'numpy.ndarray'>
Original channel shape: (4468,)
Filtered data shape: (4468,)
Processing channel F3
Original channel type: <class 'numpy.ndarray'>
Original channel shape: (4468,)
Filtered data shape: (4468,)
Processing channel F4
Original channel type: <class 'numpy.ndarray'>
Original channel shape: (4468,)
Filtered data shape: (4468,)
Processing channel C3
Original channel type: <class 'numpy.ndarray'>
Original channel shape: (4468,)
Filtered data shape: (4468,)
Processing channel C4
Original channel type: <class 'numpy.ndarray'>
Original channel shape: (4468,)
Filtered data shape: (4468,)
Processing channel P3
Original channel type: <class 'numpy.ndarray'>
Original channel shape: (4468,)
Filtered data shape: (4468,)
Processing channel P4
Original channel type: <class 'numpy.ndarray'>
Original channel sh

In [26]:
# Convert lists to arrays for further processing
alpha_powers = np.array(alpha_powers)
beta_powers = np.array(beta_powers)

# Calculate the arousal value as the ratio of beta to alpha power
arousal_values = beta_powers / alpha_powers

In [27]:
print("\nAlpha Powers:")
for i, power in enumerate(alpha_powers):
    print(f"{channel_names[i]}: {power}")

print("\nBeta Powers:")
for i, power in enumerate(beta_powers):
    print(f"{channel_names[i]}: {power}")

print("\nArousal Values:")
for i, arousal in enumerate(arousal_values):
    print(f"{channel_names[i]}: {arousal}")

print("\nMean Arousal Value:", np.mean(arousal_values))


Alpha Powers:
Fp1: 4.652537965637975
Fp2: 11.592908561489685
F3: 3.834775207948833
F4: 7.195470698192712
C3: 63.14940096404732
C4: 4.664565595371391
P3: 3.5622946802275255
P4: 4.026596152601552
O1: 9.01903429636602
O2: 7.295110204534979
F7: 15.451826101428177
F8: 15.164455231703407
T3: 3.987497421300017
T4: 11.765498870793962
T5: 11.258322360387194
T6: 432.85765937404216

Beta Powers:
Fp1: 3.568969555988046
Fp2: 7.452732427065782
F3: 2.5727777180525657
F4: 3.647049655953194
C3: 72.25113672499934
C4: 3.2524224488303934
P3: 2.237730960217621
P4: 2.506237922301197
O1: 5.914426926803535
O2: 4.336935040330514
F7: 7.3847681832242555
F8: 7.8766532802688305
T3: 3.0284426156381716
T4: 5.718924922546741
T5: 8.12936192172409
T6: 802.9335188474455

Arousal Values:
Fp1: 0.7671016512594228
Fp2: 0.642869939630414
F3: 0.6709070489241814
F4: 0.5068535206278061
C3: 1.1441301995268949
C4: 0.6972615953900927
P3: 0.6281712101579131
P4: 0.6224209797354363
O1: 0.6557716416697291
O2: 0.5944989066285076
F7: 0